In [ ]:
## If using Google colab
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)


In [3]:
import pandas as pd
import io
import numpy as np

In [19]:
# Set up read path, read file, write path, write file
curr_project_name = "project_erudition_emails_2"
notebook_path = "/mnt/c/Users/isaac/Desktop/mturks_assistant_github/mturks_assistant/notebooks"

input_filename = 'Batch_4367564_batch_results.xlsx'
read_path = notebook_path+"/../"+curr_project_name+"/5_mOut_clubs/"+input_filename

output_filename = '418_cleaned_mturk_health_emails.xlsx'
write_path = notebook_path+"/../"+curr_project_name+"/6_notebookProcessed_clubs/"+output_filename



,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,AutoApprovalDelayInSeconds,Expiration,NumberOfSimilarHITs,LifetimeInSeconds,AssignmentId,WorkerId,AssignmentStatus,AcceptTime,SubmitTime,AutoApprovalTime,ApprovalTime,RejectionTime,RequesterFeedback,WorkTimeInSeconds,LifetimeApprovalRate,Last30DaysApprovalRate,Last7DaysApprovalRate,Input.university_name,Input.club_name,Input.club_link,Answer.email_1,Answer.email_2,Answer.facebook,Answer.instagram,Answer.phone_number,Answer.website,Approve,Reject
0,34F34TZU83LKK4OLSTP5XBYXRNP2JW,3C3J03PXDMMF5QZJ4Y9V14SIRPTH78,Gather contact information for the student clu...,Gather all available contact information for a...,"data collection, data extraction, student club...",$0.01,Sun Mar 14 20:47:50 PDT 2021,3,BatchId:4367564;OriginalHitTemplateId:928390855;,60,259200,Wed Mar 31 20:50:04 PDT 2021,NaN,NaN,39O5D9O871EPKSCRC4RPH0MHDX0C32,A3H70RJNKHWKBD,Approved,Mon Mar 15 19:55:30 PDT 2021,Mon Mar 15 19:55:46 PDT 2021,Thu Mar 18 19:55:46 PDT 2021,Thu Mar 18 19:56:35 PDT 2021,NaN,NaN,16,100% (3016/3016),100% (2414/2414),0% (0/0),Texas A&M,Mays Healthcare Alliance,https://maroonlink.tamu.edu/organization/mha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34F34TZU83LKK4OLSTP5XBYXRNP2JW,3C3J03PXDMMF5QZJ4Y9V14SIRPTH78,Gather contact information for the student clu...,Gather all available contact information for a...,"data collection, data extraction, student club...",$0.01,Sun Mar 14 20:47:50 PDT 2021,3,BatchId:4367564;OriginalHitTemplateId:928390855;,60,259200,Wed Mar 31 20:50:04 PDT 2021,NaN,NaN,3GU1KF0O4QNDHCP8W2S1JJJ00J1BP1,A2ZKMDAQ3XUVCT,Approved,Wed Mar 17 14:24:39 PDT 2021,Wed Mar 17 14:24:52 PDT 2021,Sat Mar 20 14:24:52 PDT 2021,Sat Mar 20 14:25:07 PDT 2021,NaN,NaN,13,100% (2888/2888),100% (2021/2021),0% (0/0),Texas A&M,Mays Healthcare Alliance,https://maroonlink.tamu.edu/organization/mha,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34F34TZU83LKK4OLSTP5XBYXRNP2JW,3C3J03PXDMMF5QZJ4Y9V14SIRPTH78,Gather contact information for the student clu...,Gather all available contact information for a...,"data collection, data extraction, student club...",$0.01,Sun Mar 14 20:47:50 PDT 2021,3,BatchId:4367564;OriginalHitTemplateId:928390855;,60,259200,Wed Mar 31 20:50:04 PDT 2021,NaN,NaN,3NQL1CS15ZUMZ4FFS6FKXM6J9XYVYZ,A2XRPDH3WXT29P,Approved,Mon Mar 29 08:55:14 PDT 2021,Mon Mar 29 08:55:21 PDT 2021,Thu Apr 01 08:55:21 PDT 2021,Thu Apr 01 08:55:42 PDT 2021,NaN,NaN,7,100% (211/211),100% (211/211),0% (0/0),Texas A&M,Mays Healthcare Alliance,https://maroonlink.tamu.edu/organization/mha,g,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34KYK9TV3YU77K77YGRUQ5SOHCESB3,3C3J03PXDMMF5QZJ4Y9V14SIRPTH78,Gather contact information for the student clu...,Gather all available contact information for a...,"data collection, data extraction, student club...",$0.01,Sun Mar 14 20:47:51 PDT 2021,3,BatchId:4367564;OriginalHitTemplateId:928390855;,60,259200,Sun Mar 21 20:47:51 PDT 2021,NaN,NaN,33SA9F9TR5G18OYTIF8MCLXHWE2EWM,A2ZKMDAQ3XUVCT,Approved,Wed Mar 17 18:50:41 PDT 2021,Wed Mar 17 18:50:56 PDT 2021,Sat Mar 20 18:50:56 PDT 2021,Sat Mar 20 18:51:10 PDT 2021,NaN,NaN,15,100% (2888/2888),100% (2021/2021),0% (0/0),Texas A&M,Healthcare Finance Association,https://maroonlink.tamu.edu/organization/healt...,na,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34KYK9TV3YU77K77YGRUQ5SOHCESB3,3C3J03PXDMMF5QZJ4Y9V14SIRPTH78,Gather contact information for the student clu...,Gather all available contact information for a...,"data collection, data extraction, student club...",$0.01,Sun Mar 14 20:47:51 PDT 2021,3,BatchId:4367564;OriginalHitTemplateId:928390855;,60,259200,Sun Mar 21 20:47:51 PDT 2021,NaN,NaN,3PWWM24LH0KYN7QVRSXFW4L6HOZ82T,AR63X44649Y4Q,Approved,Mon Mar 15 10:53:58 PDT 2021,Mon Mar 15 10:54:15 PDT 2021,Thu Mar 18 10:54:15 PDT 2021,Thu Mar 18 10:54:28 PDT 2021,NaN,NaN,17,100% (552/552),0% (0/0),0% (0/0),Texas A&M,Healthcare Finance Association,https://maroonlink.tamu.edu/organization/healt...,NaN,NaN,NaN,NaN,NaN,NaN

## Read in MTurks dataframe and do preliminary reformatting and unstacking of data

In [20]:
df = pd.read_excel(read_path)

# Clean NANs
print('df nan values: ',df.isnull().sum().sum())
df = df.fillna('N/A')
print('df nan values: ',df.isnull().sum().sum())
# data.drop(columns='Unnamed: 0',inplace=True)

# Unstacking of the data side by side 
df['copy_id'] = -1

dfgroups = df.groupby(['Input.university_name','Input.club_name']).size().reset_index().rename(columns={0:'count'})

for i, x in dfgroups.iterrows():
  uni = x['Input.university_name']
  club = x['Input.club_name']
  df_zoom = df.loc[(df['Input.university_name']==uni) & (df['Input.club_name'] == club)]
    
  ctr = 1
  for j, y in df_zoom.iterrows():
    df.at[j,'copy_id'] = ctr
    ctr += 1

df2 = df.set_index(['Input.university_name','Input.club_name','copy_id'])
df2 = df2.unstack('copy_id')

# Clean NANs
print('df2 nan values: ',df2.isnull().sum().sum())
df2 = df2.fillna('N/A')
print('df2 nan values: ',df2.isnull().sum().sum())

# Removing all unnecessary columns for coming analysis
# KEEP ALL INPUT DATA FOR LATER CHECKING (all inputs, all answers, hitid, workerid)
df3 = df2[['HITId','WorkerId','Input.directory_link', 'Input.university_name', 'Input.country','Input.city','Answer.club_name','Answer.club_link',]]
print('df3 nan values: ',df3.isnull().sum().sum())
df3 = df3.fillna('N/A')
print('df3 nan values: ',df3.isnull().sum().sum())

df nan values:  93757
df nan values:  0
df2 nan values:  368568
df2 nan values:  0
df3 nan values:  0
df3 nan values:  0


In [ ]:
# get a specific column
# df3.loc[:,'Answer.email_1'][1]

mining_items = {}
mining_items['email_1'] = '@'
mining_items['email_2'] = '@'
# minint_items['facebook] = ''
# mining_items['instagram'] '@'
# mining_items['website'] = ''
# mining_items['phone_number']
redundancy = 3 # the redundancy is built into this it will take alot of work to automate this for more or less redundancy
prefix = 'Answer.'

for item in mining_items:
  print("item: ",item,' anchor: ',mining_items[item])
  anchor = mining_items[item]

  for index, row in df3.iterrows():
   # print("index ",index)
   # print("row item")
   # print(row['HITId'][1])

    
    # Case 1: All 3 are the same
    #if (row['1'+infix+item] == row['2'+infix+item] and row['1'+infix+item] == row['3'+infix+item]):
    if (row[prefix+item][1] == row[prefix+item][2] and row[prefix+item][1] == row[prefix+item][3]):
      #if ( anchor in row['1'+infix+item]):
      if ( anchor in row[prefix+item][1]):
        #row['Winner_'+item] = row['1'+infix+item]
        df3.loc[index, ('results','winner_'+item)] = row[prefix+item][1]
        #row[item+'flag'] = 'All good, triple consensus'
        df3.loc[index,('results','flag_'+item)] = 'All good, triple consensus'
        #row['Email_1_list'] = ''
        df3.loc[index,('results','list_'+item)] = ''
      else:
        #row['Winner_'+item] = 'N/A'
        df3.loc[index,('results','winner_'+item)] = 'N/A'
        #row[item+'_flag'] = 'All good, triple consensus'
        df3.loc[index,('results','flag_'+item)] = 'All good, triple consensus'
        #row[item+'_list'] = ''
        df3.loc[index,('results','list_'+item)] = ''

### checkpoint

    # Case 2: All 3 are different
    elif (row[prefix+item][1] != row[prefix+item][2] and row[prefix+item][1] != row[prefix+item][3] and row[prefix+item][2] != row[prefix+item][3]):
      templist = []
      #print(row[prefix+item][1])
      #print(row[prefix+item][2])
      #print(row[prefix+item][3])
      #print(type(row[prefix+item][3]))

      if (anchor in row[prefix+item][1]):
        templist.append(row[prefix+item][1])

      if (anchor in row[prefix+item][2]):
        templist.append(row[prefix+item][2])

      if (anchor in row[prefix+item][3]):
        templist.append(row[prefix+item][3])

      if len(templist) == 0:
        # row['winner'+item]='N/A'
        df3.loc[index,('results','winner_'+item)] = 'N/A'
        # row['flag_'+item] = 'All good, zero value'
        df3.loc[index,('results','flag_'+item)] = 'All good, zero value'
        # row['list_'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''

      if len(templist) == 1:
        # row['winner'+item]=templist[0]
        df3.loc[index,('results','winner_'+item)] = templist[0]
        # row['flag'+item] = 'Checkit, minority'
        df3.loc[index,('results','flag_'+item)] = 'Checkit, minority'
        # row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''

      if len(templist) == 2:
        # row['winner'+item] = ",".join(templist)
        df3.loc[index,('results','winner_'+item)] = ",".join(templist)
        # row['flag'+item] = 'Checkit, multiple emails'
        df3.loc[index,('results','flag_'+item)] = 'Checkit, multiple emails'
        # row['list'+item] = ''
        df3.loc[index,('results','list_'+item)] = ''
    
      if len(templist) == 3:
        df3.loc[index,('results','winner_'+item)] = ",".join(templist)
        df3.loc[index,('results','flag_'+item)] = 'Checkit, multiple emails'
        df3.loc[index,('results','list_'+item)] = ''
        
## Checkpoint


    # Case 3: 2 are the same and 1 is different
    #elif row['1'+infix+item] == row['2'+infix+item]: # implicitly, the odd one out is not equal
    elif row[prefix+item][1] == row[prefix+item][2]: # implicitly, the odd one out is not equal
      # if (anchor in row['1'+infix+item]): # there is an @ in the overlap
      if (anchor in row[prefix+item][1]): # there is an @ in the overlap
        if (anchor in row[prefix+item][3]): # if there is an @ in the odd one out, we should take a look
          #row['winner'+item] = ",".join([row[prefix+item][1],row[prefix+item][3]]) #row['1'+infix+item],row['3'+infix+item]])
          df3.loc[index,('results','winner_'+item)] = ",".join([row[prefix+item][1],row[prefix+item][3]]) #row['1'+infix+item],row['3'+infix+item]])
          # row['flag'+item] = 'Checkit, multiple emails'
          df3.loc[index,('results','flag_'+item)] = 'Checkit, multiple emails'
          # row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''

        else: # there is no @ in the odd one out
          # row['winner'+item] = row['1'+infix+item]
          df3.loc[index,('results','winner_'+item)] = row[prefix+item][1]
          #row['flag'+item] = 'All good, double consensus'
          df3.loc[index,('results','flag_'+item)] = row[prefix+item][1]
          # row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''
      
      else: # there is no @ in the two that overlap
        if anchor in row[prefix+item][3]: #row['3'+infix+item]: # if there is an @ in the odd one out, we should take a look
          # row['winner'+item] = row['3'+infix+item]
          df3.loc[index,('results','winner_'+item)] = row[prefix+item][3]
          # row['flag'+item] = 'Checkit, minority'
          df3.loc[index,('results','flag_'+item)]
          # row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''

        else: # there is no @ in the odd one out or the overlaps
          # row['winner'+item] = 'N/A'
          df3.loc[index,('results','winner_'+item)] = 'N/A'
          # row['flag'+item] = 'All good, zero value'
          df3.loc[index,('results','flag_'+item)] = 'All good, zero value'
          # row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''
      # print('hi')
    
## Checkpoint    
    # elif (row['1'+infix+item] == row['3'+infix+item]):
    elif (row[prefix+item][1] == row[prefix+item][3]):
    
      if (anchor in row[prefix+item][1]): # there is an @ in the overlap
        if (anchor in row[prefix+item][2]): # if there is an @ in the odd one out, we should take a look
          # row['winner'+item] = ','.join([row['1'+infix+item],row['2'+infix+item]])
          df3.loc[index,('results','winner_'+item)] = ','.join([row[prefix+item][1],row[prefix+item][2]])
          # row['flag'+item] = 'Checkit, multiple emails'
          df3.loc[index,('results','flag_'+item)] = 'Checkit, multiple emails'
          # row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''
        else: # there is no @ in the odd one out
          # row['winner'+item] = row['1'+infix+item]
          df3.loc[index,('results','winner_'+item)] = row[prefix+item][1]
          
          #row['flag'+item] = 'All good, double consensus'
          df3.loc[index,('results','flag_'+item)] = 'All good, double consensus'
          
          #row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''

      else: # there is no @ in the two that overlap
        if anchor in row[prefix+item][2]: # if there is an @ in the odd one out, we should take a look
          
          # row['winner'+item] = row['2'+infix+item]
          df3.loc[index,('results','winner_'+item)] = row[prefix+item][2]
          
          # row['flag'+item] = 'Checkit, minority'
          df3.loc[index,('results','flag_'+item)] = 'Checkit, minority'
          
          # row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''

        else: # there is no @ in the odd one out or the overlaps
          # row['winner'+item] = 'N/A'
          df3.loc[index,('results','winner_'+item)] = 'N/A'

          # row['flag'+item] = 'All good, zero value'
          df3.loc[index,('results','flag_'+item)] = 'All good, zero value'

          # row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''

      # print('ho')

## Checkpoint
    #elif (row['2'+infix+item] == row['3'+infix+item]):
    elif (row[prefix+item][2] == row[prefix+item][3]):
      
      #if (anchor in row['2'+infix+item]): # there is an @ in the overlap
      if (anchor in row[prefix+item][2]): # there is an @ in the overlap

        #if (anchor in row['1'+infix+item]): # if there is an @ in the odd one out, we should take a look
        if (anchor in row[prefix+item][1]): # if there is an @ in the odd one out, we should take a look
          #row['winner'+item] = ','.join([row['2'+infix+item],row['1'+infix+item]])
          df3.loc[index,('results','winner_'+item)] = ','.join([row[prefix+item][2],row[prefix+item][1]])

          #row['flag'+item] = 'Checkit, multiple emails'
          df3.loc[index,('results','flag_'+item)] = 'Checkit, multiple emails'

          #row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''

        else: # there is no @ in the odd one out
          # row['winner'+item] = row['2'+infix+item]
          df3.loc[index,('results','list_'+item)] = row[prefix+item][2]
          
          # row['flag'+item] = 'All good, double consensus'
          df3.loc[index,('results','flag_'+item)] = 'All good, double consensus'

          # row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''

## checkpoint    
      else: # there is no @ in the two that overlap
        #if anchor in row['1'+infix+item]: # if there is an @ in the odd one out, we should take a look
        if anchor in row[prefix+item][1]: #['1'+infix+item]: # if there is an @ in the odd one out, we should take a look
          #row['winner'+item] = row['1'+infix+item]
          df3.loc[index,('results','winner_'+item)] = row[prefix+item][1]

          # row['flag'+item] = 'Checkit, minority'
          df3.loc[index,('results','flag_'+item)] = 'Checkit, minority'

          # row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''

        else: # there is no @ in the odd one out or the overlaps
          #row['winner'+item] = 'N/A'
          df3.loc[index,('results','winner_'+item)] = 'N/A'

          #row['flag'+item] = 'All good, zero value'
          df3.loc[index,('results','flag_'+item)] = 'All good, zero value'

          #row['list'+item] = ''
          df3.loc[index,('results','list_'+item)] = ''

      # print('he')
    


#      print('hi')
#      print(row['1_Answer.email_1'])
#      print(row['2_Answer.email_1'])
#      print(row['3_Answer.email_1'])

    # row['Email_1_list'] = 4
    # print(row['c1'], row['c2'])

'''
'''

item:  email_1  anchor:  @
item:  email_2  anchor:  @


'\n'

In [ ]:
# get a specific column
df_emails = df3.loc[:,[('results','winner_email_1'),('results','flag_email_1'),('results','winner_email_2'),('results','flag_email_2')]]
df_emails = df_emails.fillna('N/A')
df_emails.head()

results  ...                            
copy_id                                                                        winner_email_1  ...                flag_email_2
Input.university_name        Input.club_name                                                   ...                            
Albany State University      N/A                                                          N/A  ...  All good, triple consensus
Allegany College of Maryland N/A                                                          N/A  ...  All good, triple consensus
Arizona State University     Active Minds at Arizona State University        gcarnesi@asu.edu  ...  All good, triple consensus
                             Alpha Epsilon Delta                                          N/A  ...  All good, triple consensus
                             American Medical Student Association - W  amsa.asuwest@gmail.com  ...  All good, triple consensus

[5 rows x 4 columns]

##Connect the columns together, making the database you need (and maybe keeping IDs to link the data with other data later)

In [ ]:
df_emails_large = pd.DataFrame(columns=['university','club','winner_email','flag','list_id'])
email_items = ['email_1','email_2']

for item in email_items:
  for index, row in df_emails.iterrows():
    university = index[0]
    club = index[1]
    if ',' in row[('results','winner_'+item)]:
      # create list of emails
      templist = row[('results','winner_'+item)].split(',')
      # add emails to new dataframe
      
      for email in templist:
        email = email
        flag = row[('results','flag_'+item)]
        dftemp = pd.DataFrame([[university,club,email,flag,item]],columns=['university','club','winner_email','flag','list_id'])
        # print(dftemp)
        # print('\n\n')
        df_emails_large = df_emails_large.append(dftemp)

    else:
      # add emails to new dataframe
      email= row[('results','winner_'+item)]
      flag = row[('results','flag_'+item)]
      dftemp2 = pd.DataFrame([[university,club,email,flag,item]] ,columns=['university','club','winner_email','flag','list_id'])
      # print(dftemp2)
      # print('\n\n')
      df_emails_large = df_emails_large.append(dftemp2)

'''
    print('issue')
    print(row[('results','winner_email_1')])
    print('index: ',index)
    print('\n')
'''
    # print(row[('results','winner_email_1')])

    # df_emails_large.loc[df_emails_large['list_id']=='email_2']
df_emails_large = df_emails_large.fillna('N/A')
df_emails_large = df_emails_large.loc[df_emails_large['winner_email']!='N/A']

"\n    print('issue')\n    print(row[('results','winner_email_1')])\n    print('index: ',index)\n    print('\n')\n"

## Export the dataframe

In [ ]:
df_emails_large.to_excel(write_path)